In [25]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [26]:
# Load Data
train_data = pd.read_csv("/kaggle/input/robi-datathon-2-pre-assessment/train.csv")
test_data = pd.read_csv("/kaggle/input/robi-datathon-2-pre-assessment/test.csv")
# submission file
submission_data = pd.read_csv("/kaggle/input/robi-datathon-2-pre-assessment/sample_submission.csv")

In [27]:
train_data.head(2)

<b> Data Cleaning 

In [28]:
train_data.isna().sum()

In [29]:
print(f"Train dataset shape: {train_data.shape}")
print(f"Test dataset shape: {test_data.shape}")

Drop null columns

In [30]:
train_data = train_data.drop(['s56','s57','s59'],axis=1)

In [31]:
train_data.shape

In [32]:
train_data.columns.values

In [33]:
train_data.info()

In [ ]:
# for colm in train_data:
#     print(train_data[colm].unique())

In [34]:
train_data.describe(include=["object"])

<b> Visualization

In [36]:
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
%config InlineBackend.figure_format = 'retina'

In [37]:
sns.countplot(x="s16", hue="gender", data=train_data);

In [38]:
# Feature Engineering 
numerical_feat = train_data.select_dtypes(include=np.number).columns

In [39]:
print('Numerical columns:',numerical_feat)

In [40]:

categorical_feat = train_data.select_dtypes(include='object').columns
print('Categorical columns:',categorical_feat)

In [13]:
train_data.label.value_counts()

In [41]:
import matplotlib.pyplot as plt
%matplotlib inline

# train_data.label.value_counts().plot.bar()
# plt.show()

In [42]:
for c in categorical_feat:
    if len(train_data[c].value_counts()) < 100:
        print(c)
        train_data[c].value_counts().plot.bar()
        plt.show()

In [17]:
for c in numerical_feat:
    print(c)
    train_data[c].plot.hist(bins=5)
    plt.show()

In [43]:
for c in numerical_feat:
    print(c)
    #print('min:', train_data[c].min(), 'max:', train_data[c].max())
    dropIndexes = train_data[train_data[c] > train_data[c].max()*9/10].index
    train_data.drop(dropIndexes , inplace=True)

In [44]:
for c in numerical_feat:
    print(c)
    print(train_data[c].value_counts(bins=2, sort=False))
    plt.show()

In [45]:
for c in numerical_feat:
    print(c)
    train_data[c].plot.hist(bins=100)
    plt.show()

In [46]:
fig, axes = plt.subplots(len(numerical_feat), len(numerical_feat), figsize=(16, 16), sharex=False, sharey=False)
for i in range(0,len(numerical_feat)):
    for j in range(0,len(numerical_feat)):
        axes[i,j].scatter(x = train_data[numerical_feat[i]], y = train_data[numerical_feat[j]])
fig.tight_layout()

In [54]:
# Feature Selection
target = 'label'
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score

In [55]:
numerical_features = ['s13', 's48', 'n1', 'n2', 'n3', 'n4', 'n5', 'n6', 'n7', 'n8', 'n9',
       'n10', 'n11', 'n12', 'n13', 'n14', 'n15']
categorical_features  = ['id', 'gender', 's11', 's12', 's16', 's17', 's18', 's52', 's53', 's54',
       's55', 's58', 's69', 's70', 's71']

In [56]:
model_features = numerical_features + categorical_features
# Get train data to train the pipeline
X_train = train_data[model_features]
y_train = train_data[target]

In [57]:
# Maximum number of decision trees. The effective number of trained trees can be smaller if early stopping is enabled.
NUM_TREES = 250
# Minimum number of examples in a node.
MIN_EXAMPLES = 6
# Maximum depth of the tree. max_depth=1 means that all trees will be roots.
MAX_DEPTH = 5
# Ratio of the dataset (sampling without replacement) used to train individual trees for the random sampling method.
SUBSAMPLE = 0.65
# Control the sampling of the datasets used to train individual trees.
SAMPLING_METHOD = "RANDOM"
# Ratio of the training dataset used to monitor the training. Require to be >0 if early stopping is enabled.
VALIDATION_RATIO = 0.1

In [58]:
import tensorflow as tf

In [59]:
ann = tf.keras.models.Sequential()
ann.add(tf.keras.layers.Dense(units=6, activation='relu'))
ann.add(tf.keras.layers.Dense(units=6, activation='relu'))
ann.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))
ann.compile(optimizer='adam' , loss='binary_crossentropy' , metrics=['accuracy'] )

In [60]:
ann.fit(X_train, y_train, batch_size=32, epochs=100)